## image

In [8]:
import time
import threading

import nglview as nv


In [9]:
view = nv.demo()
view

NGLWidget()

In [10]:
output_folder = '/home/yuyang/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/output'

def generate_images(v=view, output=output_folder):
    v.clear()
    v.add_cartoon(color='red')
    im0 = v.render_image()
    v.clear()
    v.add_cartoon(color='blue')
    im1 = v.render_image()
    for im in [im0, im1]:
        while not im.value:
            time.sleep(0.1)
    for n, im in zip('ab', [im0, im1]):
        with open(f'{output}/figure_{n}.png', 'wb') as fh:
            fh.write(im.value)

thread = threading.Thread(
    target = generate_images,
)
thread.daemon = True
thread.start()

## movie

### loading files

In [11]:
import os

import nglview as nv
import MDAnalysis as mda
import moviepy.editor as mpy

from MDAnalysis.tests.datafiles import PSF, DCD
from nglview.contrib.movie import MovieMaker
from time import sleep

u1 = mda.Universe(PSF, DCD)

view = nv.show_mdanalysis(u1)
view

NGLWidget(max_frame=97)

#### Use movie maler in NGL view (broken)

In [12]:
# broken, wait for update the package

# movie = MovieMaker(view, 
#                    output='my.gif', 
#                    in_memory=True)
# movie.make()


#### Generate images using thread

In [13]:
# output folder

output_folder = "/home/yuyang/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/output"

In [15]:
import threading

def generate_images(v=view, output=output_folder, total_frame=10):
    images = [] # List[bytes]

    for frame in range(0, total_frame):
        # set frame to update coordinates
        v.frame = frame
        # make sure to let NGL spending enough time to update coordinates
        sleep(0.5)
        im1 = v.render_image()
        images.append(im1)

    for im in images: # wait for browser to finish rendering
        while not im.value:
            sleep(0.1)
    
    # create _temp folder
    if not os.path.exists(f'{output}/_temp'):
        os.makedirs(f'{output}/_temp')

    for n, im in zip(range(0, total_frame), images):
        with open(f'{output}/_temp/0figure_{n}.png', 'wb') as fh:
            fh.write(im.value)

thread = threading.Thread(
    target=generate_images,
)
thread.daemon = True
thread.start()

#### Download images using `download_image` in NGL view (slow, manually)

In [ ]:
# make sure to change your web browser option to save files as default (vs open file by external program)
# NGLView will render each snapshot and save image to your web browser default download location
# uncomment all the commands below to render

# to save time for this tutorial, we make a movie with only 50 frames
for frame in range(0, 10):
    # set frame to update coordinates
    view.frame = frame
    # make sure to let NGL spending enough time to update coordinates
    sleep(0.5)
    view.download_image(filename='0figure_{}.png'.format(frame))
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(2.0)

#### Loading images

In [16]:
# In my case, my default download folder is /Users/haichit/Downloads/
template = '{}/_temp/0figure_{}.png'

# get all (sorted) image files
imagefiles = [template.format(output_folder, str(i)) for i in range(0, 10, 2)]

#### Assemble images to movie

In [18]:
# make a gif file

frame_per_second = 3
im = mpy.ImageSequenceClip(imagefiles, fps=frame_per_second)
im.write_gif(f'{output_folder}/my_movie.gif', fps=frame_per_second)


[MoviePy] Building file /home/yuyang/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/output/my_movie.gif with imageio


 83%|████████▎ | 5/6 [00:00<00:00,  9.52it/s]


#### Display video in notebook

In [19]:
# display the gif in this notebook
from IPython import display

display.HTML(f"<img src='{output_folder}/my_movie.gif'></img>")